<a href="https://colab.research.google.com/github/dvschultz/ShapeMatchingGAN/blob/master/ShapeMatchingGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Shape Matching GAN

Shape Matching GAN converts imaage textures to fit in shapes. Is it like style transfer? Kind of but no. Is it like CycleGAN? Also kind of, but no.

See [the project page](https://williamyang1991.github.io/projects/ICCV2019/SMGAN.html) for more.

I’ll show you how to load the library, run examples using the pre-trained models, and then train your own model. Be aware training can take up to the limit of Colab so you might want to save it to Google Drive just to be safe.

# Install Library and required data

In [0]:
!git clone https://github.com/dvschultz/ShapeMatchingGAN
%cd ShapeMatchingGAN/save
!gdown --id 1gjHR39deUSPChtRbKAD80waoQFTiXyMs #download example checkpoint files
!unzip ShapeMGAN-data_plus_model.zip
!rm -r data/style
!mkdir ../data/rawtext/yaheiB/train
!mv data/rawtext/yaheiB/train/*.* ../data/rawtext/yaheiB/train/
!mkdir ../data/rawtext/augment
!mv data/rawtext/augment/*.* ../data/rawtext/augment/
!mv save/*.* ./
!rmdir save
!rm ShapeMGAN-data_plus_model.zip
!rm -r data
%cd ../src

Optionally, if you want to do training I recommend connecting a Google Drive account to Colab.

In [0]:
from google.colab import drive
drive.mount('/gdrive')

# Test
Let’s run a sample test to see how it does. This will generate a file named 'output.png' in your `/output/` folder.

There are four pretrained models to train from: fire, maple leaves, water, and smoke.

In [0]:
!python test.py --scale 0.0 --structure_model ../save/fire-GS-iccv.ckpt --texture_model ../save/fire-GT-iccv.ckpt --gpu

Another example with some additional arguments. This will generate multiple images at different “scales.” It will produce something like an animation when all the frames are combined.

In [0]:
!python test.py --text_name ../data/rawtext/yaheiB/val/0801.png --scale -1 --scale_step 0.005 --structure_model ../save/fire-GS-iccv.ckpt --texture_model ../save/fire-GT-iccv.ckpt --result_dir ../output --name fire-0801 --gpu

We can zip all of these images up and download them. Once finished, in the Files tab on the left you’ll see a file named `fire_images.zip` Control-click on it and select `Download`.

In [0]:
!zip /content/fire_images.zip /content/ShapeMatchingGAN/output/fire-0801_*.png

# Train a custom model
Training a custom model happens in 3 different training parts. From what I can tell you don’t need to train the Sketch Module—the pretrained model is fine.

## Image Pre-processing
To train a custom model you need to process your image in the format SMGAN requires.

First, upload two images to Colab. The first image should be the image you want the texture/style from. The second image should be a masked (black/white) version of this image. See the examples in the `uploads` folder.

Once uploaded, run the next command with the paths for each image as arguments.

In [0]:
!python preprocess.py --image ../uploads/sample_v3.png --image_mask ../uploads/sample_mask_v3.png --max_height 400

When that cell has finished, there will be a new image in your `data/style` folder. Use this file with the `--style_name` argument when training.

## Train the Sketch Module (optional)
The Sketch Module is trained on the letter shapes. Unless you have a specific need you can use the pre-trained model for this.

In [0]:
# !python trainSketchModule.py --help
!python trainSketchModule.py --text_path ../data/rawtext/yaheiB/train --text_datasize 708 --augment_text_path ../data/rawtext/augment --augment_text_datasize 5 --batchsize 16 --Btraining_num 12800 --save_GB_name ../save/GB.ckpt --gpu

## Train the Structure Transfer model

The Structure Transfer model will take the most time to train—usually it takes a few hours to train. 

When completed there will be two new models in the `/save` folder. One is labeled {name}-GS.ckpt and the other {name}-DS.ckpt. You should download the GS as soon as its done—you don’t need the DS checkpoint unless you want to resume training at a later date.

In [0]:
!python trainStructureTransfer.py --style_name ../data/style/sample_v3.png --batchsize 16 --Straining_num 2560 --step1_epochs 30 --step2_epochs 40 --step3_epochs 80 --scale_num 4 --save_path ../save --save_name sample_v3 --gpu
#!python trainStructureTransfer.py --style_name ../data/style/ladiescrop-720.png --batchsize 16 --Straining_num 2560 --step1_epochs 30 --step2_epochs 40 --step3_epochs 80 --scale_num 4 --Sanglejitter --save_path ../save --save_name ladies --gpu

## Train the Texture Transfer
The Texture Transfer model will take the some time to train, but is much faster than the Structure model. It usually takes me an ~hour or so. 

When completed there will be two new models in the `/save` folder. One is labeled {name}-GT.ckpt and the other {name}-DT.ckpt. You should download the GT as soon as its done—you don’t need the DT checkpoint unless you want to resume training at a later date.

In [0]:
!python trainTextureTransfer.py --style_name ../data/style/sample_v3.png --batchsize 4 --Ttraining_num 800 --texture_step1_epochs 40 --save_path ../save --save_name sample_v3 --gpu

Now run a test using your custom model. make sure to reference your new GS and GT models.

In [0]:
!python test.py --text_name ../data/rawtext/yaheiB/val/0801.png --scale -1 --scale_step 0.005 --structure_model ../save/sample_v3-GS.ckpt --texture_model ../save/sample_v3-GT.ckpt --result_dir ../output --name sample_v3-0801 --gpu

In [0]:
!zip sample_v3.zip /content/ShapeMatchingGAN/output/sample_v3-0801_*.png